In [1]:
!python --version

Python 3.10.19


In [2]:
!pip install \
  tensorflow==2.15 \
  tensorflow-model-remediation \
  fairness-indicators==0.46.0 \
  tensorflow-model-analysis==0.46.0 \
  tensorflow-data-validation==1.15.1

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is co

In [5]:
# With the libraries installed, all necessary components can now be imported — including MinDiff for addressing unfair bias in models and Fairness Indicators for evaluating and improving models for fairness concerns.

import pandas as pd
import tensorflow as tf

import tensorflow_model_analysis as tfma
from google.protobuf import text_format

from tensorflow_model_remediation import min_diff

2025-11-17 23:51:20.400892: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 23:51:23.818778: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-17 23:51:32.885168: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-17 23:51:32.885438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-17 23:51:34.320788: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [6]:
# Import the dataset
acs_df = pd.read_csv(filepath_or_buffer="https://download.mlcc.google.com/mledu-datasets/acsincome_raw_2018.csv")

# Print five random rows of the pandas DataFrame.
acs_df.sample(5)

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,ST,PINCP
535863,39.0,1.0,16.0,1.0,4800.0,17.0,1.0,40.0,2.0,1.0,17.0,6500.0
1000779,38.0,3.0,20.0,5.0,9122.0,36.0,7.0,40.0,1.0,2.0,36.0,45100.0
1573071,24.0,6.0,20.0,5.0,4600.0,53.0,13.0,20.0,2.0,1.0,53.0,960.0
144984,48.0,1.0,21.0,1.0,120.0,17.0,1.0,65.0,1.0,6.0,6.0,565000.0
581384,62.0,2.0,20.0,1.0,5860.0,18.0,1.0,32.0,2.0,1.0,18.0,35000.0


In [7]:
## Change Target Value to Binary
# [As stated earlier](#scrollTo=TL5y5fY9Jy_x), the task is to predict whether the annual income of a US working adult is more than $50,000. The `PINCP` (total person's yearly income) column in the dataset represents the target variable; however, the value will need to be convereted into a binary. For each sample, an individual’s target label will be `1` if `PINCP` > `50000.0`, otherwise `0`.

LABEL_KEY = 'PINCP'
LABEL_THRESHOLD = 50000.0

acs_df[LABEL_KEY] = acs_df[LABEL_KEY].apply(
    lambda income: 1 if income > LABEL_THRESHOLD else 0)

acs_df.sample(10)

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,ST,PINCP
1153556,41.0,1.0,19.0,1.0,8320.0,25.0,0.0,40.0,2.0,1.0,39.0,0
46209,59.0,6.0,21.0,3.0,2634.0,362.0,0.0,40.0,1.0,1.0,4.0,1
710383,18.0,1.0,16.0,5.0,5240.0,2.0,17.0,25.0,2.0,5.0,25.0,0
1312178,49.0,1.0,21.0,1.0,800.0,39.0,0.0,55.0,1.0,1.0,45.0,1
68972,52.0,3.0,20.0,4.0,9121.0,5.0,0.0,10.0,1.0,2.0,5.0,0
908343,73.0,2.0,19.0,3.0,4020.0,8.0,0.0,50.0,2.0,1.0,33.0,0
1320976,23.0,1.0,16.0,5.0,6260.0,45.0,2.0,40.0,1.0,2.0,45.0,0
463645,41.0,1.0,19.0,1.0,1006.0,110.0,0.0,40.0,1.0,1.0,13.0,1
287312,50.0,1.0,21.0,5.0,1021.0,40.0,0.0,40.0,1.0,1.0,8.0,1
1273051,25.0,1.0,21.0,5.0,1650.0,42.0,2.0,45.0,2.0,1.0,42.0,1


In [8]:
## Defining Base Model
#For the purposes of this exercise, a simple, lightly-tuned [`keras.Model`](https://www.tensorflow.org/api_docs/python/tf/keras/Model) (using the [Functional API](https://www.tensorflow.org/guide/keras/functional) for preprocessing) will be created and serve as the base model for this exercise. Note that while the focus for this exercise is the technique involved in addressing fairness concerns, the model architecture — as taught throughout MLCC — would be thoughfully chosen and hyperparameter tuning would be performed before attempting to address any fairness concerns that might arise in the model.

inputs = {}
features = acs_df.copy()
features.pop(LABEL_KEY)

# Instantiate a Keras input node for each column in the dataset.
for name, column in features.items():
  if name != LABEL_KEY:
    inputs[name] = tf.keras.Input(
        shape=(1,), name=name, dtype=tf.float64)

# Stack the inputs as a dictionary and preprocess them.
def stack_dict(inputs, fun=tf.stack):
  values = []
  for key in sorted(inputs.keys()):
    values.append(tf.cast(inputs[key], tf.float64))

  return fun(values, axis=-1)

x = stack_dict(inputs, fun=tf.concat)

# Collect the features from the DataFrame, stack them together and normalize
# their values by passing them to the normalization layer.
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(stack_dict(dict(features)))

# Build the main body of the model using a normalization layer, two dense
# rectified-linear layers, and a single output node for classification.
x = normalizer(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Put it all together using the Keras Functional API
base_model = tf.keras.Model(inputs, outputs)

In [9]:
## Configuring Base Model
# Since this is a binary classification task, computing the cross-entropy loss between true labels and predicted labels will be sufficient for this exercise.

# Define the metrics used to monitor model performance while training.
METRICS = [
  tf.keras.metrics.BinaryAccuracy(name='accuracy'),
  tf.keras.metrics.AUC(name='auc'),
]

# Configure the model for training using a stochastic gradient descent
# optimizer, cross-entropy loss between true labels and predicted labels, and
# the metrics defined above to evaluate the base model during training.
base_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=METRICS)

In [10]:
## Convert to tf.data.Dataset
# Most of the exercises throughout MLCC use a [pandas DataFrame](https://developers.google.com/machine-learning/glossary/#pandas) directly as an input argument for [`Model.fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit). But in this exercise, the dataset must be converted to [`tf.data.Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) because that's the requirement for [`MinDiffModel`](https://www.tensorflow.org/responsible_ai/model_remediation/api_docs/python/model_remediation/min_diff/keras/MinDiffModel), which will be introducted later. Fortunately, this conversion is simple to do, thanks to [`tf.convert_to_tensor`](https://www.tensorflow.org/api_docs/python/tf/convert_to_tensor).
# The following helper function will be useful for preparing the dataset in subsequent code cells:

# Helper function to convert a pandas DataFrame into a tf.Data.dataset object
# necessary for the purposes of this exercise.
def dataframe_to_dataset(dataframe):
  dataframe = dataframe.copy()
  labels = dataframe.pop(LABEL_KEY)
  dataset = tf.data.Dataset.from_tensor_slices(
      ((dict(dataframe), labels)))
  return dataset

In [11]:
## Finalize Training Set & Train Base Model
# At this point, all that remains is splitting the dataset before training the base model.
# **NOTE:** *The following cell may take approximately 10—15 minutes to run.*

RANDOM_STATE = 200
BATCH_SIZE = 100
EPOCHS = 10

# Use the sample() method in pandas to split the dataset into a training set
# that represents 80% of the original dataset, then convert it to a
# tf.data.Dataset object, and finally train the model using the
# converted training set.
acs_train_df = acs_df.sample(frac=0.8, random_state=RANDOM_STATE)
acs_train_ds = dataframe_to_dataset(acs_train_df)
acs_train_batches = acs_train_ds.batch(BATCH_SIZE)

base_model.fit(acs_train_batches, epochs=EPOCHS)

Epoch 1/10
13316/13316 [==============================] - 27s 2ms/step - loss: 0.4378 - accuracy: 0.7875 - auc: 0.8645
Epoch 2/10
13316/13316 [==============================] - 22s 2ms/step - loss: 0.4296 - accuracy: 0.7925 - auc: 0.8702
Epoch 3/10
13316/13316 [==============================] - 22s 2ms/step - loss: 0.4249 - accuracy: 0.7953 - auc: 0.8735
Epoch 4/10
13316/13316 [==============================] - 21s 2ms/step - loss: 0.4203 - accuracy: 0.7983 - auc: 0.8766
Epoch 5/10
13316/13316 [==============================] - 22s 2ms/step - loss: 0.4173 - accuracy: 0.8003 - auc: 0.8785
Epoch 6/10
13316/13316 [==============================] - 21s 2ms/step - loss: 0.4150 - accuracy: 0.8020 - auc: 0.8800
Epoch 7/10
13316/13316 [==============================] - 22s 2ms/step - loss: 0.4135 - accuracy: 0.8031 - auc: 0.8810
Epoch 8/10
13316/13316 [==============================] - 21s 2ms/step - loss: 0.4127 - accuracy: 0.8038 - auc: 0.8815
Epoch 9/10
13316/13316 [========================

In [12]:
## Evaluate Base Model
# Consistent with [Ding et al.](https://proceedings.neurips.cc/paper_files/paper/2021/file/32e54441e6382a7fbacbbbaf3c450059-Paper.pdf), the overall accuracy for the base model should be at around 80% with minimal tuning and a basic model architecture. The following code cell uses the test set to evaluate the performance of the base model:

# Use the indices from the training set to create the test set, which represents
# 20% of the original dataset; then convert it to a tf.data.Dataset object, and
# evaluate the base model using the converted test set.
acs_test_df = acs_df.drop(acs_train_df.index).sample(frac=1.0)
acs_test_ds = dataframe_to_dataset(acs_test_df)
acs_test_batches = acs_test_ds.batch(BATCH_SIZE)

base_model.evaluate(acs_test_batches, batch_size=BATCH_SIZE)    

3329/3329 [==============================] - 5s 1ms/step - loss: 0.4121 - accuracy: 0.8042 - auc: 0.8823


[0.41209888458251953, 0.8041994571685791, 0.8822979927062988]

In [13]:
## Evaluating for Fairness
# With the base model trained, now would be a good opportunity to evaluate performance across demographic groups. For ease of analysis, [Fairness Indicators](https://www.tensorflow.org/responsible_ai/fairness_indicators/guide) will be used to compute fairness metrics across demographic groups and visualize results.
# To begin, a column containing all the base model's predictions from the test set will be needed in order to configure Fairness Indicators. [`model.predict(test_set)`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict) will be used to generate the output predictions.

# Generate output predictions using the test set.
base_model_predictions = base_model.predict(
    acs_test_batches, batch_size=BATCH_SIZE)

3329/3329 [==============================] - 4s 1ms/step


In [14]:
## A Note on Sensitive Attributes
# There are several features in ACSIncome that can be used to evaluate for fairness. For this exercise, the `SEX` attribute was chosen. This is in part to keep the config terse.
# The `SEX` attribute used by the [US Census Bureau surveys](https://www.census.gov/glossary/?term=Sex) to construct ACSIncome was specifically intended to capture an individual's biological sex and not gender. As such, possible ambiguity of these concepts could have tampered with their intended data collection, which could result in some misrepresentation. Ideally, there would be a separate category in such surveys that allow for an individual to express their gender identity (male, female, non-binary, agender, and so forth). But for the purposes of this exercise, the `SEX` attribute available in ACSIncome will suffice for demonstrating how to perform fairness evaluations.
# In practice, the recommended approach is to evaluate across any group that may be negatively impacted by the trained model and is accessible in the dataset, which can include race, ethnicity, working status, educational background, and even the US state that the individual is in.


SENSITIVE_ATTRIBUTE_VALUES = {1.0: "Male", 2.0: "Female"}
SENSITIVE_ATTRIBUTE_KEY = 'SEX'
PREDICTION_KEY = 'PRED'

# Make a copy of the test set, replace sensitive attribute values with
# categorial strings (for ease of visualization), and add predictions
# from the test set to the copied DataFrame as a separate column.
base_model_analysis = acs_test_df.copy()
base_model_analysis[SENSITIVE_ATTRIBUTE_KEY].replace(
    SENSITIVE_ATTRIBUTE_VALUES, inplace=True)
base_model_analysis[PREDICTION_KEY] = base_model_predictions

# Show five random examples to ensure that it looks correct.
base_model_analysis.sample(5)

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,ST,PINCP,PRED
907876,68.0,1.0,19.0,1.0,1430.0,33.0,1.0,45.0,Male,1.0,33.0,1,0.895743
309200,40.0,1.0,19.0,1.0,6230.0,25.0,1.0,40.0,Male,1.0,9.0,1,0.563241
1023819,20.0,1.0,16.0,5.0,4720.0,36.0,7.0,7.0,Female,2.0,36.0,0,0.001648
942094,48.0,1.0,23.0,2.0,51.0,34.0,0.0,50.0,Female,6.0,34.0,1,0.958496
940112,20.0,1.0,19.0,5.0,5240.0,34.0,2.0,8.0,Male,1.0,34.0,0,0.001633


In [15]:
## Configure Fairness Indicators
# With a column of predictions now included in the test set, an [`eval_config`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalConfig) must be created to use Fairness Indicators. This config must include: the names of the prediction and target label columns in the test set, a list of metrics to compute, and the sensitive attribute to designate how the metrics should be computed.
# As far as metrics goes, [there are several to choose from](https://www.tensorflow.org/tfx/model_analysis/metrics#binary_classification_metrics). For now, `ConfusionMatrixPlot` will provide everything needed to evaluate for fairness.
# **NOTE:** *The following cell may take 5—10 minutes to run.*

# Specify Fairness Indicators using eval_config.
eval_config_pbtxt = """
  model_specs {
    prediction_key: "%s"
    label_key: "%s" }
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics { class_name: "BinaryAccuracy" }
    metrics { class_name: "AUC" }
    metrics { class_name: "ConfusionMatrixPlot" }
    metrics {
      class_name: "FairnessIndicators"
      config: '{"thresholds": [0.50]}'
    }
  }
  slicing_specs {
    feature_keys: "%s"
  }
  slicing_specs {}
""" % (PREDICTION_KEY, LABEL_KEY, SENSITIVE_ATTRIBUTE_KEY)
eval_config = text_format.Parse(eval_config_pbtxt, tfma.EvalConfig())

# Run TensorFlow Model Analysis.
base_model_eval_result = tfma.analyze_raw_data(base_model_analysis, eval_config)

/home/vscode/.local/lib/python3.10/site-packages/tensorflow_model_analysis/metrics/binary_confusion_matrices.py:155: RuntimeWarning: invalid value encountered in divide
  false_omission_rate = fn / predicated_negatives


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [16]:
### Task 1: Identify Fairness Concerns
# Run the code cell below and take a moment to explore the results by selecting several metrics to display on the left pane. Individual graphs for each of the metric selected will appear in the widget to the right.
# For each individual graph, you should see a bar that represents the overall performance, followed by bars that correspond to a demogrphic group based on the sensitive attribute defined in the configuration.

# After looking at performances across different metrics,

# 1.   Is there a metric that performed equally well across demographic groups?
# 2.   Is there a metric that was disproportionate across demographic groups, despite overall performance along that metric seemed promising?

# Render Fairness Indicators.
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    base_model_eval_result)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Male', 'slice': 'SEX:Male', 'metrics': {'example_count…

In [17]:
sensitive_group_pos = acs_train_df[ ? ] # Replace the ? with a way to filter
                                        # positively labeled Female examples.

non_sensitive_group_pos = acs_train_df[ ? ] # Replace the ? with a way to filter
                                            # positively labeled Male examples.

SyntaxError: invalid syntax (732134798.py, line 1)